# LDA

In [1]:
import pandas as pd

HAS_p = pd.read_parquet('./results/HAS_p.parquet')
HAS_t = pd.read_parquet('./results/HAS_t.parquet')

In [2]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(docs):
    docs = docs.dropna()
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [3]:
def save_models_lda():
    r_time = []
    for v in HAS_p:
        model = lda(HAS_p[v])
        model['model'].save(f'./results/models_lda/{v}')
        r_time.append(model['time'])
    HAS_t['lda_training'] = pd.Series(r_time)

In [4]:
save_models_lda()

[I 2023-08-20 13:32:34,777] A new study created in memory with name: no-name-fb1bf378-76ef-4ba3-b674-8db734e3555f
[I 2023-08-20 13:32:38,710] Trial 0 finished with value: 0.15279141567222349 and parameters: {'num_topics': 22, 'alpha_categorical': 'scalar', 'eta_categorical': 'auto', 'alpha': 0.5865363891536639}. Best is trial 0 with value: 0.15279141567222349.
[I 2023-08-20 13:32:45,636] Trial 1 finished with value: 0.1546685437928849 and parameters: {'num_topics': 60, 'alpha_categorical': 'scalar', 'eta_categorical': 'auto', 'alpha': 0.045783193663668835}. Best is trial 1 with value: 0.1546685437928849.
[I 2023-08-20 13:32:49,107] A new study created in memory with name: no-name-1073e3b5-8575-4730-b113-7f445a38d37d
[I 2023-08-20 13:32:56,225] Trial 0 finished with value: 0.25213479808158307 and parameters: {'num_topics': 67, 'alpha_categorical': 'scalar', 'eta_categorical': 'scalar', 'alpha': 0.09195557123324236, 'eta': 0.6137199862398901}. Best is trial 0 with value: 0.25213479808158

In [5]:
HAS_t

,variant,segmentation,preprocess,lda_training
0,HWN,0.000000e+00,1.334094,14.322130
1,HCLWN,0.000000e+00,1.333092,12.726939
2,AWN,1.692513e+09,33.333024,27.309271
3,ACLWN,1.692513e+09,33.355028,22.753088
4,SWN,2.637917e+01,31.438030,25.117920
5,SCLWN,2.637917e+01,31.439039,26.927279


# BERTopic

In [6]:
from time import time

from bertopic import BERTopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

def save_models_bertopic():
    path_prefix = './results/models_bertopic/'
    r_time = []
    for v in HAS_p:
        t_start = time()
        docs = HAS_p[v].dropna()
        docs = [' '.join(doc) for doc in docs]
        bertopic = BERTopic(language='multilingual', embedding_model=embedding_model)
        model = bertopic.fit(docs)
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    HAS_t['bertopic_training'] = pd.Series(r_time)

In [7]:
save_models_bertopic()

In [8]:
HAS_t

,variant,segmentation,preprocess,lda_training,bertopic_training
0,HWN,0.000000e+00,1.334094,14.322130,15.278703
1,HCLWN,0.000000e+00,1.333092,12.726939,5.869205
2,AWN,1.692513e+09,33.333024,27.309271,9.887508
3,ACLWN,1.692513e+09,33.355028,22.753088,10.352225
4,SWN,2.637917e+01,31.438030,25.117920,36.455737
5,SCLWN,2.637917e+01,31.439039,26.927279,32.640802


In [9]:
HAS_t.to_parquet('./results/HAS_t.parquet')